In [5]:
# Data
import pandas as pd
import numpy as np

# Preprocessing
## sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from feature_engine.encoding import RareLabelEncoder

# Pipeline functions

In [6]:
class CategoricalGrouping(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=50):
        self.threshold = threshold
        self.categories = {}
    
    def fit(self, X, y=None):
        object_cols = X.select_dtypes(include='object').columns
        for col in object_cols:
            val_counts = X[col].value_counts()
            self.categories[col] = val_counts[val_counts >= self.threshold].index.tolist()
        return self
    
    def transform(self, X):
        X = X.copy()
        object_cols = X.select_dtypes(include='object').columns
        for col in object_cols:
            if col in self.categories:
                # Replace values below threshold and missing values with "other"
                X[col] = X[col].where(X[col].isin(self.categories[col]), "other")
        return X
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)


In [7]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mapping = {}

    def fit(self, X, y):
        self.mapping = {}
        df = pd.concat([X, y], axis=1)
        for col in X.columns:
            target_mean = df.groupby(col)[y.name].mean()
            self.mapping[col] = target_mean
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for col, target_mean in self.mapping.items():
            X_encoded[col] = X[col].map(target_mean)
        return X_encoded

In [10]:
class RemoveIdenticalColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.columns_to_drop_ = np.where(X.nunique() == 1)[0]
        return self

    def transform(self, X):
        return X.drop(columns=X.columns[self.columns_to_drop_])

In [ ]:
class RemoveWhitespace(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Data pipeline

In [8]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [ ]:
feature_list = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition']

target = 'SalePrice'

In [ ]:
X_train = train[feature_list]
y_train = train[target]

X_test = test[feature_list]
y_test = test[target]

In [15]:
cat_cols = [col for col in train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in train.columns if X_train[col].dtype in ('int64', 'float64')]

In [ ]:
target = True

if target:
    CategoricalEncoding = TargetEncoder()
else:
     CategoricalEncoding = OneHotEncoder(drop='first')



treatment_pipeline = Pipeline([
    ("numeric_imputer", SimpleImputer(strategy="mean"), num_cols),
    ("categorical_imputing", SimpleImputer(strategy="most_frequent")),
    ("remove_whitespace", RemoveWhitespace(), cat_cols),
    ("categorical_grouping", RareLabelEncoder(tol = 0.05, n_jobs = -1), cat_cols),
    ("remove_identical", RemoveIdenticalColumns()),
    ("categorical_encoding", TargetEncoder(), cat_cols),
    ("scale_features", StandardScaler(), num_cols), 
]).set_output(transform="pandas")